<a href="https://colab.research.google.com/github/soulsharp/SuperpixelGen-using-SLIC-MT/blob/main/SLIC_Algorithm_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycuda

In [2]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np
import time
import pstats
import cProfile
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

In [4]:
def initialize_cluster_centers_np(k, img_h, img_w):
  # Params needed for calculation of initial cluster center indices
  r = int(math.sqrt(k))
  c = k // r
  ratio_width =  (img_w - 10) // c
  ratio_height = (img_h - 10) // r
  row_offset = 10
  col_offset = 15

  # Determination of superpixel indices
  row_positions = ratio_height * np.arange(r, dtype=np.int32) + (row_offset * np.ones(r))
  col_positions = ratio_width * np.arange(c, dtype=np.int32) + (col_offset * np.ones(c))
  row_indices = np.broadcast_to(row_positions, (len(col_positions), len(row_positions)))
  col_indices = np.broadcast_to(col_positions, (len(row_positions), len(col_positions)))
  superpixel_indices = np.stack((row_indices, col_indices.T), axis=-1).reshape(-1, 2)

  # If k is not a perfect square, some superpixels arent generated, so we randomly generate them to get k superpixels
  num_generated = len(superpixel_indices)
  if num_generated < k:
        remaining = k - num_generated
        y_indices = np.random.randint(20, img_h - 20, remaining)
        x_indices = np.random.randint(20, img_w - 20, remaining)
        leftover_superpixels = np.stack((y_indices, x_indices), axis=-1).reshape(-1, 2)
        superpixel_indices = np.vstack((superpixel_indices, leftover_superpixels))

  return superpixel_indices

In [5]:
# calculates gradient considering all three color channels
def calc_gradient_map(img, img_h, img_w):
  grad_map = np.zeros((img_h , img_w), dtype=np.float32)
  for i in range(3):
    map = np.zeros((img_h + 2, img_w + 2), dtype=np.float32)
    np.copyto(map[1:-1, 1:-1], img[i])
    diff_map_x = map[:, 2:] - map[:, :-2]
    grad_map_x = diff_map_x[0: - 2, :] + diff_map_x[2: , :]
    grad_map = grad_map + grad_map_x

  return grad_map

In [6]:
# Function which reassign the cluster center to the pixel having the lowest gradient
def reassign_cluster_center_acc_to_grad_np(clusters,grad_map):
  y_indices = []
  x_indices = []

  # Changes the cluster index to the one having min grad value in a 3 x 3 region around it
  for cluster in clusters:
    idx_y, idx_x = int(cluster[0]), int(cluster[1])
    curr_window = grad_map[idx_y - 1 : idx_y + 2, idx_x - 1: idx_x + 2]
    min_element = np.argmin(curr_window)
    y_min = (min_element // 3) + idx_y
    x_min = (min_element % 3) + idx_x

    y_indices.append(y_min)
    x_indices.append(x_min)

  return y_indices, x_indices

In [11]:
# function to convert LAB images back to RGB and save it
def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    rgb_arr = (rgb_arr * 255).astype(np.uint8)
    io.imsave(path, rgb_arr)

In [13]:
# Reads the input RGB image
rgb = io.imread("/content/joel-filipe-QwoNAhbmLLo-unsplash.jpg")
print(rgb.shape)

# Resize images to (400 x 400) for processing
img = resize(rgb, (400,400), anti_aliasing=True)
print(img.shape)

# convert RGB to LAB
img = color.rgb2lab(img)
image = np.transpose(img, (2, 0, 1))

(4610, 3648, 3)
(400, 400, 3)


In [14]:
kernel = SourceModule(
'''
  #include<cmath>
  #include <cfloat>

// Gets the vector sum of inputVector where each row is a separate vector and also an average(refer to code for exact form of avg)
extern "C" __global__ void fusedMultiVectorSumAndAverage(float* inputVector, float* outputRow, int numRows, int numCols){

  __shared__ float accumulateSum[1024];
  int tid = threadIdx.x + blockDim.x * blockIdx.x;

  // Memory bounds check
  if(tid >= numCols) return;

  // Number of threads needed for average
  int numThreadsForAveraging = numCols / 6;
  accumulateSum[tid] = 0.0f;

  for(int i = 0; i < numRows; ++i){

    // Accesses the start of a new row each iteration
    int offset = numCols * i;

    // Accumulates sum in the first row
    accumulateSum[tid]  += inputVector[offset + tid];

  }

  __syncthreads();

  int idx = tid * 6;

  // Division takes place only for elements whose count is greater than 0
  if(tid < numThreadsForAveraging && accumulateSum[idx + 5] > 0.0){
    outputRow[idx + 0] = accumulateSum[idx + 0] / accumulateSum[idx + 5];
    outputRow[idx + 1] = accumulateSum[idx + 1] / accumulateSum[idx + 5];
    outputRow[idx + 2] = accumulateSum[idx + 2] / accumulateSum[idx + 5];
    outputRow[idx + 3] = accumulateSum[idx + 3] / accumulateSum[idx + 5];
    outputRow[idx + 4] = accumulateSum[idx + 4] / accumulateSum[idx + 5];
    outputRow[idx + 5] = 0.0f;
  }
}

extern "C" __global__ void assignClusterCenters(float* L, float* A, float* B, float* clusterPtr,
                                  int* labelMap, float* output, float* dummyOutput, int* dummyScalar, int imageWidth, int imageHeight,
                                  int numSuperpixels, int M, int S){



  extern __shared__ float sharedMem[];

  int tid = threadIdx.x;
  int global_tid = tid + blockIdx.x * blockDim.x;

  // Memory bounds check
  if(global_tid >= imageWidth * imageHeight) return;
  int Y = global_tid % imageWidth;
  int X = global_tid / imageWidth;
  int lastBlockId = (imageWidth * imageHeight + blockDim.x - 1) / blockDim.x;

  // Total number of values to load in shared memory
  int numValues = numSuperpixels * 6;

  // clusterInfo is read-only, updateBuffer accumulates sums to aggregate them later in global memory
  float* clusterInfo = sharedMem;
  float* updateBuffer = sharedMem + numValues;

  // Load clusterPtr to shared memory
  if(blockIdx.x == lastBlockId - 1){
    int numLegalThreads = imageWidth * imageHeight - (lastBlockId - 1) * blockDim.x;
    for(int i = tid; i < numValues; i+=numLegalThreads){
      clusterInfo[i] = clusterPtr[i];
      updateBuffer[i] = 0.0;
      dummyOutput[i] = clusterPtr[i];
    }
   }
  else{
    if (tid < numValues){
      clusterInfo[tid] = clusterPtr[tid];
      updateBuffer[tid] = 0.0;
    }
   }


  __syncthreads();

  float minDist = FLT_MAX;
  int minClusterIdx = 0;
  float eucDistFactor = static_cast<float>(M) / static_cast<float>(S);

  // computes distance to relevant cluster centers and assign pixel to the nearest one
  for(int i = 0; i < numSuperpixels; ++i){
    float cL = clusterInfo[i * 6 + 0];
    float cA = clusterInfo[i * 6 + 1];
    float cB = clusterInfo[i * 6 + 2];
    float cX = clusterInfo[i * 6 + 3];
    float cY = clusterInfo[i * 6 + 4];

    float absDistX = fabsf(cX - X);
    float absDistY = fabsf(cY - Y);

    // Only cluster centers in a 2S x 2S window around the pixel can influence it
    if(absDistX <= 2 * S && absDistY <= 2 * S){

      // Euclidean distance
      float eucDist = sqrtf(absDistX * absDistX + absDistY * absDistY);

      // LAB color distance
      float labDist = sqrtf((cL - L[global_tid]) * (cL - L[global_tid]) +
                      (cA - A[global_tid]) * (cA - A[global_tid]) +
                      (cB - B[global_tid]) * (cB - B[global_tid]));

      // Total distance
      float totalDist = eucDistFactor * eucDist + labDist;

      // Gets the nearest cluster's ID
      if(totalDist < minDist){
        minClusterIdx = i + 1;
        minDist = totalDist;
    }
 }

}

  // Assigns labelMap location corresponding to the tid, the min cluster ID
  labelMap[global_tid] = minClusterIdx;

  // -1 is needed because minClusterIdx goes from 1 -> numSuperpixels + 1
  int minClusterIdxStartShared = (minClusterIdx - 1) * 6 ;

  // Add values to the updateBuffer for averaging later
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 0], L[global_tid]);
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 1], A[global_tid]);
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 2], B[global_tid]);
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 3], X);
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 4], Y);
  atomicAdd(&updateBuffer[minClusterIdxStartShared + 5], 1.0);

  __syncthreads();

  // Writes updateBuffer(shared memory) to the correct location at output buffer(global memory)
  int outputRow = blockIdx.x;
  int startIdxOutputBuffer = outputRow * numValues;

  if(blockIdx.x == lastBlockId - 1){
    int numLegalThreads = imageWidth * imageHeight - (lastBlockId - 1) * blockDim.x;

    for(int i = tid; i < numValues; i+=numLegalThreads){
      output[startIdxOutputBuffer + i] = updateBuffer[i];
    }
  }

  else{

    if (tid < numValues) {
          output[startIdxOutputBuffer + tid] = updateBuffer[tid];
      }
  }

}

extern "C" __global__ void averageColorCluster(float* L, float* A, float* B, float* clusterPtr,
                                  int* labelMap, int imageWidth, int imageHeight){

  int tid = threadIdx.x;
  int global_tid = tid + blockIdx.x * blockDim.x;

  // Memory bounds check
  if(global_tid >= imageWidth * imageHeight) return;

  int label = labelMap[global_tid] - 1;

  L[global_tid] = clusterPtr[label * 6 + 0];
  A[global_tid] = clusterPtr[label * 6 + 1];
  B[global_tid] = clusterPtr[label * 6 + 2];
  }
  ''')


In [16]:
image_height = img.shape[0]
image_width = img.shape[1]
N = image_width * image_height
threadsPerBlock = 1024
numBlocks = (N + threadsPerBlock - 1) // threadsPerBlock
num_superpixels = 150
num_rows_output = np.int32(numBlocks)
num_cols_output = np.int32(num_superpixels * 6)
M = 10
S = int(math.sqrt(N /num_superpixels))
print(S, numBlocks)

32 157


In [20]:
# Gets the initial cluster center
clusters = initialize_cluster_centers_np(num_superpixels, image_height, image_width)
print(len(clusters))

# Gets the gradient map
grad_map = calc_gradient_map(image, image_height, image_width)

# Gets the X, Y coordinates of the cluster centers after perturbing them based on gradient
cols, rows = reassign_cluster_center_acc_to_grad_np(clusters, grad_map)

L = image[0][rows, cols]
A = image[1][rows, cols]
B = image[2][rows, cols]
counts = np.zeros_like(L ,np.float32)

# Constructs superpixels in the form that is expected by assignClusterCenters kernel
cluster_array = np.stack((L, A, B, rows, cols, counts), axis=-1).ravel()

150


In [21]:
L_array = img[:, :, 0].ravel().astype(np.float32)
A_array = img[:, :, 1].ravel().astype(np.float32)
B_array = img[:, :, 2].ravel().astype(np.float32)
label_array = np.zeros(image_height * image_width).astype(np.int32).ravel()
output_array = np.zeros((num_rows_output, num_cols_output), dtype=np.float32).ravel()
cluster_array = cluster_array.astype(np.float32)
dummy_output = np.ones_like(cluster_array).astype(np.float32)
dummy_scalar = np.zeros(1).astype(np.int32)
print(dummy_output.shape)

# Gets sizes needed for allocations on the GPU
size_LAB = L_array.nbytes
size_cluster_array = cluster_array.nbytes
size_label_array = label_array.nbytes
size_output_array = output_array.nbytes
size_dummy = dummy_output.nbytes
size_dummy_scalar = dummy_scalar.nbytes

# Allocates arrays on the GPU
d_l = cuda.mem_alloc(size_LAB)
d_a = cuda.mem_alloc(size_LAB)
d_b = cuda.mem_alloc(size_LAB)
d_cluster = cuda.mem_alloc(size_cluster_array)
d_label = cuda.mem_alloc(size_label_array)
d_output = cuda.mem_alloc(size_output_array)
d_dummy = cuda.mem_alloc(size_dummy)
d_dummy_scalar = cuda.mem_alloc(size_dummy_scalar)

# Copies arrays from CPU to GPU
cuda.memcpy_htod(d_l, L_array)
cuda.memcpy_htod(d_a, A_array)
cuda.memcpy_htod(d_b, B_array)
cuda.memcpy_htod(d_cluster, cluster_array)
cuda.memcpy_htod(d_label, label_array)
cuda.memcpy_htod(d_output, output_array)
cuda.memcpy_htod(d_dummy, dummy_output)
cuda.memcpy_htod(d_dummy_scalar, dummy_scalar)

assign_cluster_fn = kernel.get_function("assignClusterCenters")
update_cluster_fn = kernel.get_function("fusedMultiVectorSumAndAverage")
average_color_fn = kernel.get_function("averageColorCluster")

size_smem = 2 * size_cluster_array
print(size_smem, size_cluster_array, size_output_array)

(900,)
7200 3600 565200


In [22]:
start_time_assign_cluster_kernel = time.time()
assign_cluster_fn(d_l, d_a, d_b, d_cluster, d_label, d_output, d_dummy, d_dummy_scalar,
           np.int32(image_width), np.int32(image_height), np.int32(num_superpixels), np.int32(M), np.int32(S),
           block=(threadsPerBlock, 1, 1),
           grid = (numBlocks, 1), shared=size_smem)
cuda.Context.synchronize()
end_time_assign_cluster_kernel = time.time()
print(f"Kernel ran succesfully in {end_time_assign_cluster_kernel - start_time_assign_cluster_kernel}s")

Kernel ran succesfully in 0.0012669563293457031s


In [23]:
# Copies back to host
cuda.memcpy_dtoh(output_array, d_output)
cuda.memcpy_dtoh(label_array, d_label)
cuda.memcpy_dtoh(dummy_output, d_dummy)
cuda.memcpy_dtoh(dummy_scalar, d_dummy_scalar)

In [24]:
num_rows = np.int32(numBlocks)
num_cols = np.int32(num_superpixels * 6)
updated_cluster_array = np.zeros(num_cols).astype(np.float32)
d_input_vector = cuda.mem_alloc(size_output_array)
d_cluster_mean = cuda.mem_alloc(size_cluster_array)


In [25]:
cuda.memcpy_htod(d_input_vector, output_array)
cuda.memcpy_htod(d_cluster_mean, updated_cluster_array)

In [26]:
start_time_update_kernel = time.time()
update_cluster_fn(d_input_vector, d_cluster_mean, num_rows, num_cols,
           block=(threadsPerBlock, 1, 1),
           grid = (1, 1))
cuda.Context.synchronize()
end_time_update_kernel = time.time()

In [27]:
cuda.memcpy_dtoh(updated_cluster_array, d_cluster_mean)

In [28]:
for i in range(10):
  assign_cluster_fn(d_l, d_a, d_b, d_cluster, d_label, d_output, d_dummy, d_dummy_scalar,
           np.int32(image_width), np.int32(image_height), np.int32(num_superpixels), np.int32(M), np.int32(S),
           block=(threadsPerBlock, 1, 1),
           grid = (numBlocks, 1), shared=size_smem)


  cuda.memcpy_dtoh(output_array, d_output)

  update_cluster_fn(d_output, d_cluster, num_rows, num_cols,
           block=(threadsPerBlock, 1, 1),
           grid = (1, 1))

In [29]:
average_color_fn(d_l, d_a, d_b, d_cluster, d_label, np.int32(image_height), np.int32(image_width),
                 block=(threadsPerBlock, 1, 1),
                 grid = (numBlocks, 1))
cuda.Context.synchronize()

In [30]:
label_array_final = np.zeros_like(label_array).astype(np.int32)
L = np.zeros((image_height, image_width)).astype(np.float32)
A = np.zeros((image_height, image_width)).astype(np.float32)
B = np.zeros((image_height, image_width)).astype(np.float32)

In [32]:
cuda.memcpy_dtoh(label_array_final, d_label)
cuda.memcpy_dtoh(cluster_array, d_cluster)
cuda.memcpy_dtoh(L, d_l)
cuda.memcpy_dtoh(A, d_a)
cuda.memcpy_dtoh(B, d_b)

In [33]:
lab_img = np.stack([L, A, B], axis=-1)

In [54]:
path = "results.png"
lab2rgb(path, lab_img)